<h1>Resultados Númericos Aplicando Diferencias Finitas</h1>
<center>Sebastián Marroquin</center>


El problema de la ecuación de Onda ahora será discretizado en el espacio y el tiempo por un método de diferencia finita.

### Discretización del Dominio.

El dominio temporal $[0, T]$ está representado por un número finito de puntos en la malla:

\begin{split}t_0=0 < t_1 < t_2 < \cdots < t_{N-1} < t_N = T \thinspace .\end{split}

De manera similar, el dominio espacial $[0, L]$ se reemplaza por un conjunto de puntos de malla:

\begin{split}0 = x_0 < x_1 < x_2 < \cdots < x_{N_x-1} < x_{N_x} = L \thinspace .\end{split}

Uno puede ver la malla como bidimensional en el plano x, t, que consiste en puntos $(x_i, t_n) = i=0,\ldots,N_x, n=0,\ldots,N$

### Cumpliendo la ecuación en los puntos de malla.

Para una solución numérica por el método de diferencias finitas, relajamos la condición de que la ecuación de onda cumple en todos los puntos del dominio $(0, L) × (0, T)$ para el requisito de que se cumpla el PDE en los puntos de malla:

$\frac{\partial^2}{\partial t^2} u(x_i, t_n) =
     c^2\frac{\partial^2}{\partial x^2} u(x_i, t_n), (1)$
     
para $i = 1, ...,$ $Nx − 1$ y $n = 1, ..., N.$ Para $n = 0$ tenemos las condiciones iniciales $u = I (x)$ y $ut = 0$, y en los límites $i = 0$, $Nx$ tenemos la condición de límite $u = 0$.

### Sustitución de derivados por diferencias finitas.

Las derivadas de segundo orden pueden ser reemplazados por diferencias centrales. La diferencia aproximada más utilizada de la derivada de segundo orden es:
$
\frac{\partial^2}{\partial t^2}u(x_i,t_n)\approx
\frac{u_i^{n+1} - 2u_i^n + u^{n-1}_i}{\Delta t^2} = [D_tD_t u]^n_i, $

y una definición similar de la derivada de segundo orden en la dirección x:  $\frac{\partial^2}{\partial x^2}u(x_i,t_n)\approx
\frac{u_{i+1}^{n} - 2u_i^n + u^{n}_{i-1}}{\Delta x^2} = [D_xD_x u]^n_i
\thinspace.$

Ahora podemos reemplazar los derivados en (1) y obtener:

$\frac{u_i^{n+1} - 2u_i^n + u^{n-1}_i}{\Delta t^2} =
     c^2\frac{u_{i+1}^{n} - 2u_i^n + u^{n}_{i-1}}{\Delta x^2},$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Parametros de la Onda
c = 10.0 	# Velocidad de la Onda
time = 10.0 # Tamaño del Dominio (frontera)
L = 4.0 	# Tamaño del espacio del Dominio

# Parametros de las Diferencias Finitas
nx = 201 			# Espacio Discreto
sigma = 0.8 
dx = L / nx 		# Tamaño de paso
dt = sigma*dx/c 	# Tiempo de paso
nt = int(time/dt) 	# Tamaños discretizados del intervalo

# Parametros iniciales
m = 3 	# nodos en las condiciones iniciales
A = 1.0 # Amplitud de onda

# Devolvemos el valor para u
def u0Init(nx,A,L,m):
	x = np.linspace(0,L,nx)
	stand = A*np.sin(m*np.pi*x/L)
	gauss = A*np.exp(-10*(x-0.25*L)**2) # Forma gaussiana
	u0 = stand + gauss
	return u0

# retorna el valor inicial de  u_t
def u_t0Init(nx,A,L,m):
	x = np.linspace(0,L,nx)
	u_t0 = np.zeros(nx) # No hay velocidad inicial
	return u_t0


# Declaramos una funcion pra la animacion
def animate(data):
	x = np.linspace(0,L,nx)
	y = data
	line.set_data(x,y)
	return line,

# Aplicamos las diferencias finitas
def wave1D(nt,dt,dx,c,L,A,m):
	un = np.zeros((nt,nx)) 		# Solucion 
	u0 = u0Init(nx,A,L,m)  		# posicion inicial
	u_t0 = u_t0Init(nx,A,L,m) 	# velovidad inicial
	rSq = (c*dt/dx)**2 			# combinacion del tamaño de paso

	# Primer paso, para contar la velocidad inicial
	un[0,:] = u0
	un[1,1:-2] = un[0,1:-2] - dt*u_t0[1:-2] + 0.5*rSq*(un[0,2:-1]-2*un[0,1:-2]+un[0,0:-3])
	un[1,0] = 0; un[1,-1] = 0; # Condiciones de frontera
	
	# Resto de los pasos 
	for i in range(1,nt-1):
		un[i+1,1:-2] = 2*un[i,1:-2]-un[i-1,1:-2] + \
				rSq*(un[i,2:-1]-2*un[i,1:-2]+un[i,0:-3])

		un[i+1,0] = 0 # Condiciones de forntera
		un[i+1,-1] = 0

	return un


fig = plt.figure()
ax = plt.axes(xlim=(0,L),ylim=(-1.5*A,1.5*A))
line, = ax.plot([],[],lw=2)

un = wave1D(nt,dt,dx,c,L,A,m)

anim = animation.FuncAnimation(fig,animate,frames=un,interval=20)
anim.save('waveFD.html')
plt.show()